In [ ]:
--https://services3.arcgis.com/bWPjFyq029ChCGur/arcgis/rest/services/Transmission_Line/FeatureServer/2/query?outFields=*&where=1%3D1&f=geojson

--ALLOW ACCESS TO WEBSITE TO QUERY
CREATE OR REPLACE NETWORK RULE arcgis_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('services3.arcgis.com');

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION ARCGIS_Access_integration
  ALLOWED_NETWORK_RULES = (arcgis_rule)
  ENABLED = true;


In [ ]:
--PYTHON FUNCTION TO READ URLS. THE HEADER INFORMATION IS THERE TO BYPASS PYTHON DENIAL FROM CDN

-- CDN is content delivery network
--    To successfully bypass CDN restrictions, you would typically include headers in the request, such as:

--    User-Agent: Mimicking a real browser's user-agent string so that the server thinks the request is coming from a legitimate web browser rather than a script.
--    Referer: The referer header might be included to make the request look like it is coming from a legitimate page.
--    Authorization: Some CDNs might require an API key or token for access, especially if the content is restricted.

 


CREATE OR REPLACE FUNCTION get_url(url string)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.10
HANDLER = 'get_url'
EXTERNAL_ACCESS_INTEGRATIONS = (ARCGIS_Access_integration)
PACKAGES = ('snowflake-snowpark-python','requests')
AS
$$
import _snowflake
import requests
import json
session = requests.Session()
def get_url(url):

    response = session.get(url)
    return response.text
$$;

In [ ]:
-- As is the url is one big "record" or blob
-- PARSE_JSON() is primarily used when you have a JSON string (as text) and want to convert it into a VARIANT so that you can easily query or manipulate the data inside it, using Snowflake’s built-in functions for working with semi-structured data.

-- CEC table created below is just one record


create or replace table cec as
select parse_json(get_url('https://services3.arcgis.com/bWPjFyq029ChCGur/arcgis/rest/services/Transmission_Line/FeatureServer/2/query?outFields=*&where=1%3D1&f=geojson'))::variant data;


In [ ]:

-- in the json from data we are selecting 
--    data is assumed to be the variant column
--    features is an array, we are selecting the first element
--    geometry is the field we are selecting from features
--
-- the result of > select data:features[0].geometry geom from cec
--     is one record named geom, 
--
-- try_to_geometry
--     is used to attempt to convert a given value (such as a string) into a geospatial geometry object.
--     A string that represents a Well-Known Text (WKT) representation of a geometry object (such as POINT, LINESTRING, POLYGON, etc.) or any other valid geometry format. It can be a string literal, column, or any other expression that results in a string.
--     looks like this cell is testing out what we can do

-- as seen in the result of this cell is a bunch of lat long coordinates



with cte as (
select data:features[0].geometry geom from cec)
select try_to_geometry(geom::string) geom
from cte

In [ ]:
-- The result of this step is view with bunch of LINESTRING records each with a number of coordinates
-- 
-- f.value:geometry::string
--    f.value is accessing the individual values in the array stored in cec.data:feature
--
--    The TO_GEOMETRY() function is used to convert the string representation of a geometry (e.g., GeoJSON-like format) into an actual Snowflake geometry object (like POINT, LINESTRING, POLYGON, etc.).
--
-- The ST_ASWKT() function converts the geometry object into a Well-Known Text (WKT) representation, which is a textual representation of a geometry.
--
--
-- The FLATTEN function is used to expand a nested array (in this case, cec.data:features
-- The LATERAL keyword allows you to reference the preceding row (i.e., the row from cec) inside the FLATTEN function.

create or replace view transmissions_sample AS 
select ST_ASWKT(to_geometry(f.value:geometry::string)) geom
from cec,
lateral flatten(input => cec.data:features) as f
limit 500;
select * from transmissions_sample limit 5;

In [ ]:
# Let's see a H3 map of our data, we chose a level of 5, update as appropriate or make it dynamic
# Hover over a cell and see the number of expired registrations
#
import streamlit as st
import pandas as pd
import pydeck as pdk
from snowflake.snowpark.context import get_active_session
session = get_active_session()

df = session.table('transmissions_sample').to_pandas()

# Process WKT to extract coordinates (assuming POINT geometries for simplicity)
import shapely.wkt as wkt

def extract_coordinates(wkt_geom):
    geom = wkt.loads(wkt_geom)
    if geom.geom_type == 'Point':
        return [(geom.x, geom.y)]
    elif geom.geom_type in ['MultiPoint', 'LineString', 'Polygon']:
        return list(geom.coords)
    elif geom.geom_type in ['MultiLineString', 'MultiPolygon']:
        coords = []
        for part in geom.geoms:
            coords.extend(part.coords)
        return coords
    else:
        return []

# Extract coordinates
df['coordinates'] = df['GEOM'].apply(extract_coordinates)

# Flatten the coordinates into separate rows for visualization
flattened_data = []
for idx, row in df.iterrows():
    for coord in row['coordinates']:
        flattened_data.append({'latitude': coord[1], 'longitude': coord[0]})

map_data = pd.DataFrame(flattened_data)

# Define pydeck visualization
layer = pdk.Layer(
    'ScatterplotLayer',
    data=map_data,
    get_position='[longitude, latitude]',
    get_radius=100,
    get_fill_color='[255, 0, 0, 200]',
    pickable=True,
)

view_state = pdk.ViewState(
    longitude=map_data['longitude'].mean(),
    latitude=map_data['latitude'].mean(),
    zoom=10,
    pitch=50,
)

# mapbox API key
mbkey = ''

# define map details - layers, map provider, initial state, tooltip
deck_all_layers = pdk.Deck(
        map_provider = 'mapbox',
        api_keys = {"mapbox":mbkey},
        map_style = 'mapbox://styles/mapbox/light-v11',
        layers=[
             layer
               ], 
        tooltip={"text": "{latitude}, {longitude}"},
        initial_view_state=pdk.ViewState(
            #Sac 38.5781° N, 121.4944° W
            latitude=38.5781,
            longitude=-121.4944,
            zoom=7,
            pitch=30
            )
        )

st.write("transmission lines")

# Display the map
st.pydeck_chart(deck_all_layers)